# Baseline Surface Radiation Network (BSRN)

The [Baseline Surface Radiation Network (BSRN)](https://bsrn.awi.de/) is a global network of high-quality solar radiation monitoring stations under the [World Climate Research Programme (WCRP)](https://www.wcrp-climate.org/) {cite:p}`driemel_baseline_2018`.

According to the [World Radiation Monitoring Center (WRMC)](https://bsrn.awi.de/project/objectives/):
> The data [from the BSRN stations] are of primary importance in supporting the validation and confirmation of satellite and computer model estimates of these quantities. At a small number of stations (currently 74 in total, 58 active) in contrasting climatic zones, covering a latitude range from 80°N to 90°S, solar and atmospheric radiation is measured with instruments of the highest available accuracy and with high time resolution.

All BSRN stations are required to meet the basic [station requirements](http://bsrn.awi.de/en/stations/join-bsrn/). A list of activate, inactive, and candidate BSRN stations can be retrieved from the SolarStations [station listing](../station_listing) and are shown below.

In [1]:
import pandas as pd

stations = pd.read_csv('solarstations.csv', sep=';', encoding='latin1')
stations = stations[stations['Network'].str.contains('BSRN')]
stations['Time period'] = stations['Time period'].astype(str).replace('nan','')
stations

,Station full name,Abbreviation,State,Country,Latitude,Longitude,Elevation,Time period,Network,Unnamed: 9,Comment,URL,Tier 2,Instrument
0,Abashiri,ABS,NaN,Japan,44.0178,144.2797,38.0,2021-,BSRN,NaN,Candidate station (no. 86),NaN,1,Thermopile
1,Alert,ALE,Lincoln Sea,Canada,82.4900,-62.4200,127.0,2004-2014,BSRN,NaN,NaN,NaN,1,Thermopile
2,Alice Springs,ASP,Northern Territory,Australia,-23.7980,133.8880,547.0,1995-,BSRN,NaN,NaN,NaN,1,Thermopile
3,Barrow,BAR,Alaska,USA,71.3230,-156.6070,8.0,1992-,BSRN,NaN,NaN,https://gml.noaa.gov/obop/brw/,1,Thermopile
4,Bermuda,BER,Bermuda,USA,32.2670,-64.6670,8.0,"1992-2013, 2016-",BSRN,NaN,NaN,NaN,1,Thermopile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Tiruvallur,TIR,NaN,India,13.0923,79.9738,36.0,2014-,BSRN,NaN,NaN,NaN,1,Thermopile
75,Terra Nova Bay,TNB,NaN,Antarctica,-74.6223,164.2283,28.0,,BSRN,NaN,Candidate Station (no. 89),https://gml.noaa.gov/grad/meetings/BSRN2018_do...,1,Thermopile
76,Toravere,TOR,NaN,Estonia,58.2540,26.4620,70.0,1999-,BSRN,NaN,NaN,NaN,1,Thermopile
77,Xianghe,XIA,NaN,China,39.7540,116.9620,32.0,2005-2016,BSRN,NaN,"Station obstructed, no longer in BSRN since 2016",NaN,1,Thermopile


```{margin} Station metadata
Click the plus symbol above to see a table of the stations and their metadata.
```

In [10]:
import folium
from folium import plugins

EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"

# Create Folium map
m = folium.Map(
    location=[0, 15],
    zoom_start=1, min_zoom=1, max_bounds=True,
    control_scale=True,  # Adds distance scale in lower left corner
    tiles='openstreetmap',
)

# Function for determining station color
def icon_color(row):
    if row['Time period'].endswith('-'):  # active station
        color = 'green'
    elif row['Time period'] == '':  # candidate station
        color = 'gray'
    else:  # inactive/closed station
        color = 'red'
    return color

# Add each station to the map
# Consider using apply instead of for loop to add stations in case of many stations
for index, row in stations.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station full name'] + ' - ' + str(row['State']) + ' ' + row['Country'],
        tooltip=row['Abbreviation'],
        icon=folium.Icon(color=icon_color(row), icon='bolt', prefix='fa')
    ).add_to(m)

folium.raster_layers.TileLayer(EsriImagery, name='World imagery', attr=EsriAttribution).add_to(m)
folium.LayerControl(position='topright').add_to(m)

# Additional options and plugins
# Note it's not possible to change the position of the scale
plugins.MiniMap(toggle_display=True, zoom_level_fixed=1, minimized=True, position='bottomright').add_to(m)  # Add minimap to the map
plugins.Fullscreen(position='topright').add_to(m)  # Add full screen button to map
folium.LatLngPopup().add_to(m)  # Show latitude/longitude when clicking on the map
# plugins.LocateControl(position='topright').add_to(m)  # Add button for your position
# plugins.MeasureControl(position='topleft').add_to(m)  # Add distance length measurement tool


# Add Category Legend
legend_html = """
<div style="position:fixed;
     top: 10px; 
     left: 10px; 
     width: 120px; 
     height: 85px; 
     border:2px solid grey; 
     z-index: 9999;
     font-size:14px;">
     &nbsp;<b>Station markers</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp;Active<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp;Inactive<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:gray"></i>&nbsp;Candidate<br>
</div>"""

m.get_root().html.add_child(folium.Element(legend_html))  # Add Legend

# Show the map
m

## Station requirements
As a mimimum a BSRN station is required to measure global horizontal irradiance (GHI), direct normal irradiance (DNI), diffuse horizontal irradiance (DHI), 

Additional metadata may be found at the [BSRN website](https://wiki.pangaea.de/wiki/BSRN#Sortable_Table_of_Stations) and in the individual data files (e.g., horizon profile).

```{note}
Unlike the majority of solar radiation monitoring networks, the BSRN website does not have a subpage for each station (with photos, etc.). This would have been very useful when assessing the usage of the station, for example in regards to the potential impact of nearby structures, etc. Note a few photos of the BSRN stations can be found [here](https://bsrn.awi.de/other/picture-gallery/). There station log books are also not available. It should also be noted that the files on the FTP server do not include wind speed and direction. 
```

## Data retrieval

Data from the BSRN stations is stored in monthly files for each station and can be freely downloaded either via [FTP](https://bsrn.awi.de/?id=387) or [Pangea](https://bsrn.awi.de/data/data-retrieval-via-pangaea/). Credentials for accessing the BSRN FTP server can be obtained as described in the [data release guidelines](https://bsrn.awi.de/data/conditions-of-data-release).

```{admonition} Data release guidelines
Please read the [BSRN data release guidelines](https://bsrn.awi.de/data/conditions-of-data-release/) before using any data and make sure to properly cite the BSRN.
```

```{warning}
WRMC highly recommends that all users do their own quality checks of the data after extracting BSRN-data!
```

The data can also be downloaded programmatically using the [pvlib-python](https://pvlib-python.readthedocs.io) library, specifically the [`get_bsrn`](https://pvlib-python.readthedocs.io/en/latest/generated/pvlib.iotools.get_bsrn.html) function. An example of how to use pvlib to download two months of data from the Cabauw (CAB) station is shown below:

In [2]:
import os
bsrn_username = os.environ["BSRN_FTP_USERNAME"]
bsrn_password = os.environ["BSRN_FTP_PASSWORD"]

NameError: name 'os' is not defined

In [ ]:
import pvlib

df, meta = pvlib.iotools.get_bsrn(
    station='CAB',  # three letter code for the Cabauw station
    start=pd.Timestamp(2018,6,1),
    end=pd.Timestamp(2018,7,14),
    username=bsrn_username,  # replace with your own username
    password=bsrn_password,  # replace with your own password
)

df.head(12)

```{margin} Available parameters
Click the plus symbol above to see the first 12 data entries.
```

For a description of the input parameters, see the [pvlib documentation](https://pvlib-python.readthedocs.io/en/latest/generated/pvlib.iotools.get_bsrn.html).

```{admonition} Retrieving BSRN data in R
R users can find similar functionality in the [SolarData](https://github.com/dazhiyang/SolarData) R package.
```

The data retrieved from all BSRN stations includes measurements of the three irradiance components, as well as longwave downwelling irradiance, temperature humidity, etc.

A few of the parameters in the datasets for the month of data are visualized below.

In [ ]:
axes = df[['ghi','dni','dhi','lwd','temp_air']].plot(
    subplots=True, legend=False, rot=0, figsize=(8,8), sharex=True)

# Set y-labels and y-limits
axes[0].set_ylabel('GHI [W/m$^2$]'), axes[0].set_ylim(-10,1300)
axes[1].set_ylabel('DNI [W/m$^2$]'), axes[1].set_ylim(-10,1300)
axes[2].set_ylabel('DHI [W/m$^2$]'), axes[2].set_ylim(-10,1300)
axes[3].set_ylabel('LWD [W/m$^2$]'), axes[4].set_ylim(200,500)
_ = axes[4].set_ylabel('Temperature [°]'), axes[4].set_ylim(0,40)

Notice how that they are multiple periods where there is gaps in the irradiance data.

## To do
* Make a list of bad stations
* What is the difference in available parameters between FTP and Pangea?
* Add continent to solarstations.csv
* Add description of minimum requirements for BSRN stations
* Make note that delay of a few years is not uncommon and give link to where they can see the status of the data.

### Done
* Cite the BSRN
* Add legend for colors
* Get correct year of operation from here: https://dataportals.pangaea.de/bsrn/
* Have different color for active and inactive stations


# References

```{bibliography}
:filter: docname in docnames
```